In [1176]:
import numpy as np
import sklearn
import pandas as pd
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import ensemble,tree,linear_model
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler, StandardScaler
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier,GradientBoostingRegressor)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, accuracy_score
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split,cross_val_score
# from sklearn.cross_validation import KFold
import tensorflow as tf
import warnings
import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls
py.init_notebook_mode(connected=True)
%reload_ext autoreload
%autoreload 2
%matplotlib inline
warnings.filterwarnings('ignore')

pd.options.display.max_columns=99

In [1]:
Input_Data = pd.read_csv('data/TADPOLE_InputData.csv')
Input_Data.head()

NameError: name 'pd' is not defined

In [1181]:
# format date, drop constant columns + null PTID rows
def Input_prep(data):
    data['EXAMDATE'] = pd.to_datetime(data['EXAMDATE'], errors='coerce')
    data['EXAMDATE_bl'] = pd.to_datetime(data['EXAMDATE_bl'])
    
    # We will takeoff all columns where we have a unique value (constants)
    # It is useful because this columns don't give us none information
    discovering_consts = [col for col in data.columns if data[col].nunique() == 1]
    # printing the total of columns dropped 
    print(len(discovering_consts),"columns are dropped ")
    # Get the shape of the processed dataset
    data = data.drop(discovering_consts,axis=1)
    print("After dropping constants, the shape of the data set is:",data.shape)
    
    data=data.dropna(subset=['PTID_Key'])
    print("After dropping missing PTID_Key, the shape of the dataset is: ", data.shape)
    return data


In [1182]:
# Drop most null columns, threshold in %
def drop_maj(data, threshold):
    total = data.isnull().sum().sort_values(ascending = False) # getting the sum of null values and ordering
    percent = (data.isnull().sum() / data.isnull().count() * 100 ).sort_values(ascending = False) #getting the percent and order of null
    df = pd.concat([total, percent], axis=1, keys=['Total', 'Percent']) # Concatenating the total and percent
    most_nan= [idx for idx in df[~(df['Total'] == 0)].index if df[~(df['Total'] == 0)].loc[idx,'Percent']>threshold]
    print("There are ", len(most_nan),"columns with ",threshold, "% missing values")
    data = data.drop(most_nan,axis=1)
    print("After dropping most null columns, the shape of the dataset is: ", data.shape)
    
    return data


In [1183]:
Input_Data=Input_prep(Input_Data)
Input_Data = drop_maj(Input_Data,60)

TypeError: tuple indices must be integers or slices, not str

In [ ]:
Input_Data['DX_bl'].isnull().sum()

In [ ]:
list = np.unique(Input_Data['DXCHANGE'].values)
list

In [ ]:
# Get diagnosis of every visit based on DX_bl and DXCHANGE, then keep DX and drop the other two
def get_DX(Input_new):
    idx0 = Input_new.loc[Input_new['DXCHANGE'].isin([-1,1, 2, 3])].index.values 
    for ix0 in idx0:
            Input_new.loc[ix0,'DX'] = Input_new.loc[ix0,'DX_bl']

    idx1 = Input_new.loc[Input_new['DXCHANGE'].isin([4, 8])].index.values 
    for ix1 in idx1:
            Input_new.loc[ix1,'DX'] = 'EMCI'

    idx2 = Input_new.loc[Input_new['DXCHANGE'].isin([5, 6])].index.values 
    for ix2 in idx2:
            Input_new.loc[ix2,'DX'] = 'AD'

    idx3 = Input_new.loc[Input_new['DXCHANGE'].isin([7, 9])].index.values 
    for ix3 in idx3:
            Input_new.loc[ix3,'DX'] = 'CN'
    Input_new['DX']=Input_new['DX'].fillna(Input_new['DX_bl'])        
    print(Input_new['DX_bl'].isnull().sum())
    print(Input_new['DXCHANGE'].isnull().sum())
    print(Input_new['DX'].isnull().sum().sum())
    list = np.unique(Input_new['DX'].values)
    print(list)
    d={'SMC': 'CN','CN':'CN','AD':'AD','EMCI': 'MCI','LMCI':'MCI','MCI':'MCI'}
    Input_new['DX']=Input_new['DX'].map(d)
    print(Input_new['DX'].isnull().sum().sum())
    list = np.unique(Input_new['DX'].values)
    print(list)
    
#     Input_new['DX']=Input_new['DX'].rename_categories({'SMC': 'CN', 'EMCI': 'MCI','LMCI':'MCI'})
    Input_new.head()
    
    return Input_new

In [ ]:
Input=get_DX(Input_Data)
print(Input.shape)
Input=Input.drop(['DX_bl','DXCHANGE'],axis=1)
print(Input.shape)

### Filling missing values

In [ ]:
# Perform linear interpolation, then ffil and bfil
def linear_interp(data):
    ID_list = np.unique(data.PTID_Key.values)
    # Create an empty dataframe with all columns from data
    Input_new=pd.DataFrame(columns=data.columns)
#     print(ID_list)
    for ID in ID_list:
        if ID/100==0:
            print(ID/len(ID_list), "% done processing")
        df=data[data['PTID_Key']==ID]
        # interpolate only for numeric data
        df=df.interpolate()
        # ffill, bfill numeric data that can't be interpolate as well as categorical data
        df=df.fillna(method='ffill')
        df=df.fillna(method='bfill')
        Input_new=pd.concat([Input_new, df], ignore_index=True)

    print("Remaining missing values: ", Input_new.isnull().sum().sum() )
    print("Filled percentage: ", (1- Input_new.isnull().sum().sum()/data.isnull().sum().sum())*100,"%")
    return Input_new

In [ ]:
def interp(data):
    # Sort data by PTID_Key and EXAMDATE
    data = data.sort_values(by=['PTID_Key','EXAMDATE'])
    
    # Total missing values, if want to get missing values of each column, delete the last sum()
    print("Total missing values: ", data.isnull().sum().sum())
    
    # Linear interpolation, then ffil, bfil
    data_new=linear_interp(data)
#     print(data_new.head())
    
    # Fill the rest with median of each DX_bl category
    # Map DX_bl values to numbers first
    # look how many different values 'DX_bl' have
#     print('Different DX_bl values:',np.unique(data_new['DX_bl'].values))

#     # map those values into numbers
#     DXbl_mapping = {"CN": 0, "EMCI": 1, "LMCI": 2, "SMC": 3,"AD": 4}
#     data_new['DX_bl'] = data_new['DX_bl'].map(DXbl_mapping)
    return data_new

In [ ]:
Input_proc=interp(Input)
Input=Input_proc
Input_proc.head()

In [ ]:
# Sort data by PTID and EXAMDATE, perform linear interpolatioin, then ffil, bfill, 
# then fill the rest with the same DX_bl category
def fill_null(data_new):

#     data_new = data_new.groupby(['DX_bl']).transform(lambda x: x.fillna(x.median()))
#     print("Total number of missing values: ", data_new.isnull().sum().sum())
#     transformed = data_new.groupby(['DX_bl']).transform(lambda x: x.fillna(x.value_counts().index[0]))
    null_col=data_new.columns[data_new.isnull().any()].tolist()
    categ=data_new.select_dtypes(exclude=["number","datetime"])
    categ_col=categ.columns[categ.isnull().any()].tolist()
    num=data_new.select_dtypes(exclude=["object","datetime"])
    num_col=num.columns[num.isnull().any()].tolist()
    print("Number of columns with missing values is: ",len(null_col))
    print("Number of numerical columns with missing values is: ",len(num_col))
    print("Number of categorical columns with missing values is: ",len(categ_col))
    # Create an empty dataframe with all columns from data_data
    print(data_new.shape)
    for col in num_col:
        # progress report
#         print(col)
#         data_new[col]=data_new.groupby("DX_bl").transform(lambda x: x.fillna(x.median()))
        data_new[col] = data_new.groupby(['DX'])[col].transform(lambda x: x.fillna(x.mean()))
    for col2 in categ_col:
#         data_new[col]=data_new.groupby("DX_bl").transform(lambda x: x.fillna(x.median()))
        data_new[col2] = data_new.groupby(['DX'])[col2].transform(lambda x: x.fillna(x.value_counts().index[0]))
    
#     col=data_new.columns[data_new.isnull().any()].tolist()
#     data_new=data_new.drop(col,axis=1)
    print("Total number of missing values: ", data_new.isnull().sum().sum())
    print("shape of the dataset: ",data_new.shape)
    return data_new

In [ ]:
Input_new=fill_null(Input)

In [ ]:
Input_new.head()
Input_new['DX']

In [ ]:
Input_new.to_csv('data/Input_interp_filledCat.csv',index=False)

In [ ]:
# Category columns type is: object
# Input_new['PTID_Key'] = Input_new['PTID_Key'].astype(float)
categ_col=Input_new.select_dtypes(exclude=["number","datetime"]).columns.tolist()
print(categ_col)
# categ_col.remove('EXAMDATE')
# categ_col.remove('EXAMDATE_bl')

for col in categ_col:
    Input_new[col]=Input_new[col].astype('category')
    Input_new[col] = Input_new[col].cat.codes
Input_new.head()

In [ ]:
Input_new.to_csv('data/Input_interp_filledCat_codedCat.csv',index=False)

In [ ]:
from sklearn.preprocessing import StandardScaler
def scale(data):
#     df=pd.DataFrame(columns=data.columns)
#     print(df.shape)
#     df=data.drop(['PTID_Key','EXAMDATE', 'EXAMDATE_bl'], axis=1)
    features=data.select_dtypes(exclude=["datetime"]).columns.tolist()
    # Separating out the features
    for feature in features:
    # Standardizing the features
        data[feature] = StandardScaler().fit_transform(data[feature].values.reshape(-1,1))
    # df = StandardScaler().fit_transform(df)
    return data
    # df.isnull().sum().sum()

In [ ]:
data=Input_new.drop(['PTID_Key','EXAMDATE', 'EXAMDATE_bl','DX'], axis=1)
data.head()
df=scale(data)
df.shape

In [1185]:
from sklearn.decomposition import PCA
# Make an instance of the Model
pca = PCA(.95)
# pca = PCA(n_components=50)
y=pca.fit_transform(df)
principalDf = pd.DataFrame(data = y
             )
principalDf.shape

(8715, 278)

In [1186]:
pca.explained_variance_ratio_.cumsum()

array([0.18438938, 0.30051561, 0.34447506, 0.38079047, 0.40337519,
       0.42236031, 0.43893658, 0.45207206, 0.46465974, 0.4758954 ,
       0.48543784, 0.49481925, 0.50351574, 0.51191496, 0.51949153,
       0.52681555, 0.53394366, 0.54068007, 0.5472531 , 0.55354567,
       0.55970446, 0.56560171, 0.57137235, 0.57694395, 0.58242909,
       0.58759372, 0.5926046 , 0.59747942, 0.6022031 , 0.60682614,
       0.6113599 , 0.61574445, 0.6200073 , 0.62424559, 0.62843499,
       0.63256738, 0.63660521, 0.64050721, 0.64432325, 0.64800929,
       0.65168472, 0.65528058, 0.65881717, 0.66234034, 0.66569269,
       0.66901605, 0.6722483 , 0.67546121, 0.67860518, 0.68171516,
       0.6847968 , 0.68786711, 0.6908093 , 0.69370288, 0.69656202,
       0.69936323, 0.70215706, 0.70490732, 0.70764138, 0.71034426,
       0.7130223 , 0.71565529, 0.71826149, 0.72083868, 0.72335261,
       0.72583556, 0.72829479, 0.73069909, 0.73309754, 0.73548331,
       0.73784971, 0.74016237, 0.74246057, 0.74473842, 0.74696

In [1187]:
Input_new.shape
principalDf=scale(principalDf)

In [1188]:
principalDf['PTID_Key']=Input_new['PTID_Key']
principalDf['EXAMDATE']=Input_new['EXAMDATE']
principalDf['DX']=Input_new['DX']
principalDf.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,...,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,PTID_Key,EXAMDATE,DX
0,-0.914682,-0.601231,-1.025777,0.646176,1.011424,0.545620,0.413133,0.157729,0.240342,1.240933,-0.614915,1.991610,0.506536,0.278690,0.153647,-0.377177,1.514802,-0.019620,0.283497,-0.656795,0.221551,0.350008,0.196467,-0.604921,-1.521657,-0.185724,-1.876779,-0.208486,0.673291,-0.596495,0.631191,0.029072,0.877182,-0.778522,0.761286,-1.076368,-0.941719,0.219719,2.491332,-1.257230,-0.303478,-1.353673,-0.387802,0.508279,-0.721791,-0.805647,-0.692598,0.739660,0.774142,...,-1.111548,-1.504085,0.280236,-1.026748,-0.935085,-0.300665,-1.065586,-0.794485,0.810877,0.240799,-0.403455,-0.313444,-0.471123,1.063699,-0.801149,-0.386264,0.054835,0.332211,-2.399556,1.094873,2.336983,-2.151991,0.262187,0.434412,1.312646,-1.291214,0.415570,-1.990970,-1.362115,-0.359844,-0.691137,0.531001,0.796599,0.538899,0.668736,-0.362717,0.526381,-1.574725,1.199216,0.881033,-0.162544,1.078744,1.362077,1.059333,0.267904,-0.928759,1.0,2010-12-10,2
1,-0.551814,-0.979159,-1.117663,1.076771,0.625453,0.594147,0.200665,-0.109799,0.167253,1.085187,-0.908410,0.744116,0.728125,-0.159216,0.144665,0.513853,1.799283,-0.349975,0.227961,-0.650050,0.099265,0.732234,0.033733,-0.569960,-1.853381,0.309617,-1.390592,-0.524115,0.653087,0.003768,1.020155,-0.361017,1.143764,-0.811648,0.879657,-0.934078,-0.719017,-0.174933,1.807723,-0.246274,0.197087,-0.588264,-0.028376,1.524079,-0.491840,-0.773806,-0.905032,0.587289,0.482128,...,-1.362359,-0.384883,-0.396091,-1.114439,-1.497867,0.073814,-1.047930,-0.472296,0.541844,-0.597084,-0.788613,-0.759914,-0.559378,0.918252,-0.153937,-0.072157,0.764843,0.574582,-1.299033,1.859001,0.993939,-0.972449,1.815612,-0.805537,1.022320,-1.829569,0.756406,-0.444980,-2.338430,0.632880,-0.912492,1.094919,0.033690,0.674762,0.643271,-0.362390,0.543195,-1.660143,1.020524,1.949152,-0.949106,-0.750425,0.614402,1.703636,-0.747710,-0.966335,1.0,2011-04-07,2
2,-0.819792,-0.982411,-1.010481,1.050002,0.690168,0.794730,0.281609,-0.584068,0.522935,1.314278,-0.468990,1.015674,0.095780,-0.090131,0.121122,0.641263,1.195369,-0.043699,0.300918,-0.510973,0.033367,0.266118,0.286762,-0.413685,-1.853315,0.160174,-1.505841,-0.255259,1.073686,-0.130857,0.113276,-0.490394,0.355425,-1.346322,0.936334,-1.610474,-0.638695,-0.203596,1.830329,-0.354029,0.131519,-1.169916,0.146008,1.354220,0.128852,-0.306639,-0.527814,0.150390,0.098864,...,-2.125737,-0.956597,0.800094,-0.947476,-1.217942,-0.685731,0.343844,0.290111,-0.847178,-0.035952,0.708540,0.377110,0.002219,1.057412,0.114082,-0.368786,-0.383575,0.110381,-1.250354,-0.132147,0.602319,-0.741663,1.161139,0.772682,0.659802,-0.370310,0.504005,-1.390165,-1.299680,-0.612009,-1.319248,-0.379710,0.861513,0.097416,1.025970,0.005399,0.394066,-0.682618,1.512006,0.429191,-0.094298,0.603369,0.183042,0.871928,0.794578,0.589846,1.0,2011-09-08,2
3,-1.336588,0.124982,0.265170,0.702093,-0.806993,0.915643,-0.873193,0.532319,-1.443681,0.450639,-0.561673,-0.045912,0.461204,-0.131562,0.077892,-0.862214,0.354069,-0.058075,-0.284119,-2.005307,-0.728140,-0.122163,0.388809,-0.725412,-0.164666,-0.481027,-0.053525,1.137394,-0.665674,-0.085696,0.019830,-0.143032,0.536505,-0.038190,0.683802,0.057937,-0.216116,-0.079253,0.046781,-0.709941,0.157998,0.185480,0.370402,-0.753654,0.781240,0.886344,-0.357192,-1.061215,0.452014,...,0.714254,-0.367555,-1.874418,-0.115493,-0.505355,1.326566,0.931132,-1.040033,0.000094,-0.922823,0.459259,0.136026,0.516101,-1.122543,-1.276170,0.537032,-0.185670,-0.037686,0.163978,1.703618,-0.464324,-0.760336,0.292739,-0.525850,1.064035,-1.654064,-0.097459,-0.879186,0.406081,-0.507800,-0.140618,0.504883,1.182153,1.805809,-0.803499,0.384380,-0.600851,0.237532,2.262692,1.360018,0.069672,0.544021,-1.105401,

In [1189]:
principalDf.to_csv('data/Input_pca_0.95thres.csv',index=False)

In [921]:
# sort the correlation coefficients in descending order
def check_corr(data):
    c = data.corr().abs()

    s = c.unstack()
    so = s.sort_values(ascending=False).drop_duplicates()
    print(so)

In [924]:
def del_corr(dataset, threshold):
    col_corr = [] # Set of all the names of deleted columns
    corr_matrix = dataset.corr().abs()
    print("Before deleting > ", threshold*100, "% correlated columns, the shape of the dataset is: ", dataset.shape)
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (corr_matrix.iloc[i, j] >= threshold) and (corr_matrix.columns[j] not in col_corr):
                colname = corr_matrix.columns[i] # getting the name of column
                col_corr.append(colname)
                proc_data=dataset.drop(col_corr,axis=1)
    print("After deleting > ", threshold*100, "% correlated columns, the shape of the dataset is: ", proc_data.shape)
    return proc_data
baseline_features = ['PTID_Key','EXAMDATE','Month','AGE','Hippocampus_bl','Ventricles_bl',
                 'WholeBrain_bl','ADAS13_bl','ICV_bl','FAQ_bl','RAVLT_immediate_bl',
                            'ADAS11_bl','MMSE_bl','CDRSB_bl']
print(Input_new.shape)
data_toproc=Input_new.drop(baseline_features,axis=1)
print(data_toproc.shape)
Input_remove_corr=del_corr(data_toproc,0.7)
Input_remove_corr=pd.concat([Input_remove_corr, Input_new[baseline_features]], axis=1)
print(Input_remove_corr.shape)
Input_remove_corr.head()
# check_corr(Input_proc)


(8715, 716)
(8715, 702)
Before deleting >  70.0 % correlated columns, the shape of the dataset is:  (8715, 702)
After deleting >  70.0 % correlated columns, the shape of the dataset is:  (8715, 10)
(8715, 24)


,DX_bl,DXCHANGE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,PTMARRY,CDRSB,EXAMDATE_bl,Years_bl,PTID_Key,EXAMDATE,Month,AGE,Hippocampus_bl,Ventricles_bl,WholeBrain_bl,ADAS13_bl,ICV_bl,FAQ_bl,RAVLT_immediate_bl,ADAS11_bl,MMSE_bl,CDRSB_bl
0,1,1.0,1,13.0,1,6,1,1.50,2010-12-10,0.000000,1.0,2010-12-10,0.0,81.6,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,27.0,1.5
1,1,1.0,1,13.0,1,6,1,2.25,2010-12-10,0.323066,1.0,2011-04-07,3.0,81.6,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,27.0,1.5
2,1,1.0,1,13.0,1,6,1,3.00,2010-12-10,0.744695,1.0,2011-09-08,6.0,81.6,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,27.0,1.5
3,4,2.0,1,16.0,1,6,1,4.50,2006-07-21,0.000000,2.0,2006-07-21,0.0,76.6,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,25.0,4.5
4,4,2.0,1,16.0,1,6,1,5.00,2006-07-21,0.490075,2.0,2007-01-16,6.0,76.6,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,25.0,4.5


In [925]:
Input_proc.to_csv('data/Input_remove_corr.csv',index=False)

#### Detecting outliers

In [ ]:
# from scipy import stats
# import numpy as np
# z = np.abs(stats.zscore(df))
# # print(z)
# threshold = 3
# print(df.shape)
# # print("Before removing outliers, z-score higher than 3 is: ", np.where(z > 4))
# print(z[82][9])
# df_o = df[(z < 4).all(axis=1)]
# # df_o2=scale(df_o)
# print(df_o.shape)
# z_z = np.abs(stats.zscore(df_o))
# print(z_z[43][6])
# print("After removing outliers, z-score higher than 3 is: ", np.where(z_z > 4))